In [12]:
import netCDF4 # packages to open 'netcdf' file
import numpy as np # numpy and pandas packages to pre-process the dataset
import pandas as pd

import matplotlib.pyplot as plt # to create plots and graphs
from mpl_toolkits.basemap import Basemap # to create geo-spatial map, requires dependencies installation
import plotly.express as px

from bs4 import BeautifulSoup # to load webcontent and retrieve data from link
from IPython.display import display, HTML
from IPython import display
from datetime import datetime

In [13]:
## Libraries
from urllib import request, parse
from http.cookiejar import CookieJar
import getpass
import netrc
import os
import requests
import json
import pprint
# from osgeo import gdal
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import rasterio
#from rasterio.plot import show
import numpy as np
import os
import time
from netCDF4 import Dataset
import netCDF4
from owslib.wms import WebMapService
%matplotlib inline

### Earthdata Login Authentication
An Earthdata Login account is required to access data from NASA EOSDIS. In order to access data from the Harmony API, you will need to create an account in the Earthdata Login UAT environment. Please visit https://uat.urs.earthdata.nasa.gov to set up an account in this test environment. These accounts, as all Earthdata Login accounts, are free to create and only take a moment to set up.

We need some boilerplate up front to log in to Earthdata Login. The function below will allow Python scripts to log into any Earthdata Login application programmatically. To avoid being prompted for credentials every time you run and also allow clients such as curl to log in, you can add the following to a .netrc (_netrc on Windows) file in your home directory:

machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
Make sure that this file is only readable by the current user or you will receive an error stating "netrc access too permissive."

$ chmod 0600 ~/.netrc

In [15]:
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

### Now call the above function to setup Earthdata Login for subsequent requests

In [14]:
netrc.netrc(r'C:\Users\Ochibobo\_netrc')

machine uat.urs.earthdata.nasa.gov
	login Mitchell_Odili
	password 01BDMEL$#&a19003

In [16]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

Please provide your Earthdata Login credentials to allow data access
Your credentials will only be passed to uat.urs.earthdata.nasa.gov and will not be exposed in Jupyter
Username:Mitchell_Odili
········


## Identify a data collection of interest
- In this example, the data source for downloading the OCO2 and OCO3 data will 
be redirected to https://oco2.gesdisc.eosdis.nasa.gov/data// website
- Filtered and searched data will are redirected on the root directory 'oco2.gesdisc'

## FORMAT strucutre of the file
### OCO2 data downloads from geodisc

* https://oco2.gesdisc.eosdis.nasa.gov/data//
* platform/Instrument:
    - OCO2_DATA/
* Level:
    - OCO2_L2_Standard.10r
    
* Year:
    - /2021
    - /365/

* Filename:
    - oco2_L2StdGL_39897a_211231_B10206r_220224223435.h5

## Loading the web contents to locate the data sources/links

### Example: Retrieving a data from source
### Level:
- OCO2_L2_Lite_FP.10r
- https://oco2.gesdisc.eosdis.nasa.gov/data//OCO2_DATA/OCO2_L2_Lite_FP.10r/
- Year: 2019

In [17]:
params={
    'platform': "OCO2_DATA",
    'Level': "OCO2_L2_Lite_FP.10r",
    'Year': '2019',
}

source_url= "https://oco2.gesdisc.eosdis.nasa.gov/data//"

# passing parameter
earth_data_url= source_url + '/{platform}/{Level}/{Year}/'.format( **params)
earth_data_url

'https://oco2.gesdisc.eosdis.nasa.gov/data///OCO2_DATA/OCO2_L2_Lite_FP.10r/2019/'

In [18]:
cmr_response= requests.get(earth_data_url)

print("Request response: ", cmr_response)

Request response:  <Response [200]>


* Scrapping the web page content from the source OCO2 data source 'geodisc'

In [19]:
content= BeautifulSoup(cmr_response.text, 'lxml')

* Storing all the links from the specified webpage
* Includes the data sources and xml files

In [21]:
all_links= []
for j in content.find_all('a', href=True):
    print(j['href'])
    all_links.append(j['href'])

https://www.nasa.gov
https://disc.gsfc.nasa.gov/data-access
https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Download%20Data%20Files%20from%20HTTPS%20Service%20with%20wget
https://urs.earthdata.nasa.gov/approve_app?client_id=e2WVk8Pw6weeLUKZYOxvTQ
/data/OCO2_DATA/OCO2_L2_Lite_FP.10r/
/data/OCO2_DATA/OCO2_L2_Lite_FP.10r/
oco2_LtCO2_190101_B10206Ar_200729172616s.nc4
oco2_LtCO2_190101_B10206Ar_200729172616s.nc4
oco2_LtCO2_190101_B10206Ar_200729172616s.nc4.xml
oco2_LtCO2_190101_B10206Ar_200729172616s.nc4.xml
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4.xml
oco2_LtCO2_190102_B10206Ar_200729172942s.nc4.xml
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4.xml
oco2_LtCO2_190103_B10206Ar_200729173012s.nc4.xml
oco2_LtCO2_190104_B10206Ar_200729173043s.nc4
oco2_LtCO2_190104_B10206Ar_200729173043s.nc4
oco2_LtCO2_

oco2_LtCO2_190608_B10206Ar_200729195938s.nc4
oco2_LtCO2_190608_B10206Ar_200729195938s.nc4.xml
oco2_LtCO2_190608_B10206Ar_200729195938s.nc4.xml
oco2_LtCO2_190609_B10206Ar_200729200041s.nc4
oco2_LtCO2_190609_B10206Ar_200729200041s.nc4
oco2_LtCO2_190609_B10206Ar_200729200041s.nc4.xml
oco2_LtCO2_190609_B10206Ar_200729200041s.nc4.xml
oco2_LtCO2_190610_B10206Ar_200729200121s.nc4
oco2_LtCO2_190610_B10206Ar_200729200121s.nc4
oco2_LtCO2_190610_B10206Ar_200729200121s.nc4.xml
oco2_LtCO2_190610_B10206Ar_200729200121s.nc4.xml
oco2_LtCO2_190611_B10206Ar_200729200210s.nc4
oco2_LtCO2_190611_B10206Ar_200729200210s.nc4
oco2_LtCO2_190611_B10206Ar_200729200210s.nc4.xml
oco2_LtCO2_190611_B10206Ar_200729200210s.nc4.xml
oco2_LtCO2_190612_B10206Ar_200729200218s.nc4
oco2_LtCO2_190612_B10206Ar_200729200218s.nc4
oco2_LtCO2_190612_B10206Ar_200729200218s.nc4.xml
oco2_LtCO2_190612_B10206Ar_200729200218s.nc4.xml
oco2_LtCO2_190613_B10206Ar_200729200349s.nc4
oco2_LtCO2_190613_B10206Ar_200729200349s.nc4
oco2_LtCO2_1906

oco2_LtCO2_191110_B10206Ar_200729223124s.nc4
oco2_LtCO2_191110_B10206Ar_200729223124s.nc4.xml
oco2_LtCO2_191110_B10206Ar_200729223124s.nc4.xml
oco2_LtCO2_191111_B10206Ar_200729223355s.nc4
oco2_LtCO2_191111_B10206Ar_200729223355s.nc4
oco2_LtCO2_191111_B10206Ar_200729223355s.nc4.xml
oco2_LtCO2_191111_B10206Ar_200729223355s.nc4.xml
oco2_LtCO2_191112_B10206Ar_200729223445s.nc4
oco2_LtCO2_191112_B10206Ar_200729223445s.nc4
oco2_LtCO2_191112_B10206Ar_200729223445s.nc4.xml
oco2_LtCO2_191112_B10206Ar_200729223445s.nc4.xml
oco2_LtCO2_191113_B10206Ar_200729223500s.nc4
oco2_LtCO2_191113_B10206Ar_200729223500s.nc4
oco2_LtCO2_191113_B10206Ar_200729223500s.nc4.xml
oco2_LtCO2_191113_B10206Ar_200729223500s.nc4.xml
oco2_LtCO2_191114_B10206Ar_200729223523s.nc4
oco2_LtCO2_191114_B10206Ar_200729223523s.nc4
oco2_LtCO2_191114_B10206Ar_200729223523s.nc4.xml
oco2_LtCO2_191114_B10206Ar_200729223523s.nc4.xml
oco2_LtCO2_191115_B10206Ar_200729223640s.nc4
oco2_LtCO2_191115_B10206Ar_200729223640s.nc4
oco2_LtCO2_1911

### Data cleaning: OCO2 data source links only
- Next steps: 
    * storing the html links for individual data source
    * Making requests to get access to data source

In [22]:
data_source_links=[]

for k in range(0, len(all_links)):
    if all_links[k].endswith(".nc4"):
        # Strip the 'html' from the links 
        data_source_links.append(all_links[k]) #.strip('.xml'))

In [23]:
data_source_links[0]

'oco2_LtCO2_190101_B10206Ar_200729172616s.nc4'

In [25]:
# my_df= netCDF4.Dataset(data_source_links[0])